# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-DU bypasses F1AP Setup procedure when receiving UEContextModificationRequest, and starts gtpu instance directly.
Repro Steps:
1) Attacker sends an incorrect UE Context Setup Request to the gNB-CU.
2) gNB-CU rejects the request as expected.
3) Attacker immediately sends a UE Context Setup Success.
4) In parallel, attacker sends an initialRRCTransfer with mutated DU ID and Transaction ID.
5) Attacker follows with an RRC Setup Success message.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-DU bypasses required F1AP Setup/UE Context control flow when a crafted sequence arrives: after a rejected UE Context Setup request at CU, attacker sends UE Context Setup Success and, in parallel, an Initial UL RRC Transfer with mutated DU ID and Transaction ID, followed by an RRC Setup Success. DU proceeds to start GTP-U handling without proper F1AP setup handshake and binding of IDs.",
    "interface_guess": ["F1AP", "RRC", "GTP-U"],
    "procedure_guess": [
      "F1 Setup",
      "UE Context Setup",
      "UE Context Modification",
      "Initial UL RRC Message Transfer",
      "Downlink/Uplink RRC Message Transfer",
      "RRC Setup / RRC Setup Complete"
    ],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "unknown (mutated by attacker)",
      "cu_ue_f1ap_id": "unknown",
      "du_ue_f1ap_id": "unknown",
      "du_id": "mutated by attacker",
      "rnti": "unknown",
      "pci": "unknown",
      "served_cell_list": []
    },
    "signals_or_timers": [
      "F1AP Setup precondition",
      "UE context establishment precondition",
      "ID binding: CU-UE-F1AP-ID / DU-UE-F1AP-ID",
      "Transaction scope for UE Context procedures"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["Initial UL RRC Message Transfer", "UE-associated logical F1-connection"],
      "title": ["8.4.1 Initial UL RRC Message Transfer", "9.2.3.1 INITIAL UL RRC MESSAGE TRANSFER"]
    },
    {
      "Keywords": ["UE Context Modification", "Request/Response", "directionality"],
      "title": ["8.3.4 UE Context Modification procedure", "9.2.2.x UE CONTEXT MODIFICATION messages"]
    },
    {
      "Keywords": ["UE Context Setup", "preconditions", "failure handling"],
      "title": ["8.3.2 UE Context Setup procedure", "9.2.2.x UE CONTEXT SETUP messages"]
    },
    {
      "Keywords": ["Transaction ID", "procedure scoping"],
      "title": ["9.3.1.23 Transaction ID"]
    },
    {
      "Keywords": ["gNB-CU UE F1AP ID", "gNB-DU UE F1AP ID", "UE context binding"],
      "title": ["9.3.1.4 gNB-CU UE F1AP ID", "9.3.1.5 gNB-DU UE F1AP ID"]
    }
  ],
  "expected_behaviour": "DU shall complete F1 Setup before any UE-associated procedures. Initial UL RRC Message Transfer shall initiate the UE-associated logical F1-connection. UE Context Setup/Modification must use valid, bound CU/DU UE F1AP IDs and a consistent Transaction ID per procedure and direction. DU must not start user-plane (GTP-U/F1-U) handling or accept context modifications before successful setup and proper ID/transaction binding."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "F1AP/f1ap_du_interface_management.c",
      "function_name": "DU_send_F1_SETUP_REQUEST",
      "reason": "Initiates F1 Setup from DU; precondition for UE-associated procedures."
    },
    {
      "path": "F1AP/f1ap_du_rrc_message_transfer.c",
      "function_name": "DU_send_INITIAL_UL_RRC_MESSAGE_TRANSFER",
      "reason": "Sends Initial UL RRC Message Transfer; per spec this initiates UE-associated logical F1-connection."
    },
    {
      "path": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER",
      "reason": "CU-side decode/handling; links to RRC and UE context creation."
    },
    {
      "path": "F1AP/f1ap_du_ue_context_management.c",
      "function_name": "DU_handle_UE_CONTEXT_SETUP_REQUEST",
      "reason": "DU handling of UE Context Setup Request; should enforce ID binding and state."
    },
    {
      "path": "F1AP/f1ap_du_ue_context_management.c",
      "function_name": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST",
      "reason": "DU handling of UE Context Modification Request; potential entry for bypass if guards missing."
    },
    {
      "path": "LAYER2/nr_pdcp/cucp_cuup_handler.c",
      "function_name": "e1_bearer_context_modif",
      "reason": "Updates F1-U GTP tunnel on DL UP TNL Info; shows when DU-side GTP params are applied."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§8.4.1.2 Initial UL RRC Message Transfer — Successful operation",
      "text": "The gNB-DU initiates the procedure by sending an INITIAL UL RRC MESSAGE TRANSFER. The establishment of the UE-associated logical F1-connection shall be initiated as part of the procedure."
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.3.1.23 Transaction ID",
      "text": "The Transaction ID uniquely identifies a procedure among all ongoing parallel procedures of the same type initiated by the same peer. Messages belonging to the same procedure use the same Transaction ID."
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.3.1.4 / §9.3.1.5 UE F1AP IDs",
      "text": "gNB-CU UE F1AP ID and gNB-DU UE F1AP ID uniquely identify the UE association over F1 within CU and DU respectively; used to bind messages to a single UE context."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_du_rrc_message_transfer.c",
      "location": "L74–L100",
      "text": "DU_send_INITIAL_UL_RRC_MESSAGE_TRANSFER encodes INITIAL UL RRC MESSAGE TRANSFER and sends via f1ap_itti_send_sctp_data_req(assoc_id, buffer, len); errors logged if encoding fails."
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_du_ue_context_management.c",
      "location": "L125–L137",
      "text": "DU_handle_UE_CONTEXT_MODIFICATION_REQUEST decodes UE Context Modification Request and forwards to ue_context_modification_request(&req); returns error if decode fails."
    },
    {
      "kind": "code",
      "source": "LAYER2/nr_pdcp/cucp_cuup_handler.c",
      "location": "L340–L345",
      "text": "Within e1_bearer_context_modif, for each DL UP TNL param, GtpuUpdateTunnelOutgoingAddressAndTeid(f1inst, ue_id, drb_id, addr, teid) updates F1-U tunnel only when DL UP params are provided."
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-DU", "to": "gNB-CU", "message": "F1 SETUP REQUEST", "precond": "NoF1Setup", "postcond": "F1SetupInProgress"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "F1 SETUP RESPONSE", "precond": "F1SetupInProgress", "postcond": "F1Ready"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "INITIAL UL RRC MESSAGE TRANSFER (UL-CCCH + DU->CU RRC info)", "precond": "F1Ready", "postcond": "UEConnInit"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT SETUP REQUEST (IDs, SRB1, DL UP TNL if any)", "precond": "UEConnInit", "postcond": "UEContextEstablished"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT SETUP RESPONSE", "precond": "UEContextEstablished", "postcond": "UEContextEstablished"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "DL RRC MESSAGE TRANSFER (RRCSetup)", "precond": "UEContextEstablished", "postcond": "UEContextEstablished"},
    {"from": "UE", "to": "gNB-DU", "message": "UL RRC (RRCSetupComplete) -> UL RRC MESSAGE TRANSFER", "precond": "UEContextEstablished", "postcond": "UEContextEstablished"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UE CONTEXT MODIFICATION REQUEST (DL UP TNL updates)", "precond": "UEContextEstablished", "postcond": "UEContextModifying"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE CONTEXT MODIFICATION RESPONSE", "precond": "UEContextModifying", "postcond": "UEContextEstablished"}
  ],
  "state_machine": {
    "states": ["NoF1Setup", "F1SetupInProgress", "F1Ready", "UEConnInit", "UEContextEstablished", "UEContextModifying"],
    "transitions": [
      {"from": "NoF1Setup", "to": "F1SetupInProgress", "on": "F1 SETUP REQUEST", "guard": "stream==0"},
      {"from": "F1SetupInProgress", "to": "F1Ready", "on": "F1 SETUP RESPONSE", "guard": "decode ok"},
      {"from": "F1Ready", "to": "UEConnInit", "on": "INITIAL UL RRC MESSAGE TRANSFER", "guard": "TransactionID set; NR CGI + C-RNTI present"},
      {"from": "UEConnInit", "to": "UEContextEstablished", "on": "UE CONTEXT SETUP REQUEST/RESPONSE", "guard": "valid CU/DU UE F1AP IDs bound"},
      {"from": "UEContextEstablished", "to": "UEContextModifying", "on": "UE CONTEXT MODIFICATION REQUEST", "guard": "same TransactionID within procedure"},
      {"from": "UEContextModifying", "to": "UEContextEstablished", "on": "UE CONTEXT MODIFICATION RESPONSE", "guard": "IDs consistent; applied"},
      {"from": "any", "to": "UEContextEstablished", "on": "DL/UL RRC MESSAGE TRANSFER", "guard": "only if UE-associated logical F1 exists"}
    ]
  },
  "assumptions": [
    "DU rejects UE-associated messages if F1 Setup not completed",
    "UE CONTEXT SETUP RESPONSE originates only from DU upon CU request",
    "Transaction ID integrity is enforced per procedure",
    "GTP-U/F1-U parameters applied only after valid UE context established"
  ],
  "questions": [
    "Is there an explicit guard in DU to check F1 Ready before handling UE CONTEXT MODIFICATION REQUEST?",
    "Are mutated DU ID / Transaction ID detected before applying DL UP TNL updates?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "F1 Setup completed before UE-associated procedures",
      "result": "UNKNOWN",
      "evidence": ["Spec §8.4.1.2: UE-associated F1-connection shall be initiated with Initial UL RRC", "Code F1AP/f1ap_du_ue_context_management.c:L125–L137 shows no explicit F1-Ready guard"]
    },
    {
      "name": "Initial UL RRC establishes UE-associated logical F1-connection",
      "result": "PASS",
      "evidence": ["Spec §8.4.1.2", "Code F1AP/f1ap_du_rrc_message_transfer.c:L74–L100 send path present"]
    },
    {
      "name": "Directionality of UE Context Setup/Modification respected",
      "result": "PASS",
      "evidence": ["Spec 9.2.2.x tables (REQUEST/RESPONSE directions)", "DU handlers exist for requests; responses sent from opposite side"]
    },
    {
      "name": "Transaction ID scoping per procedure",
      "result": "UNKNOWN",
      "evidence": ["Spec §9.3.1.23 defines scoping", "No explicit Transaction ID validation found in DU handlers snippet"]
    },
    {
      "name": "CU/DU UE F1AP ID binding enforced",
      "result": "PASS",
      "evidence": ["Spec §9.3.1.4/§9.3.1.5", "Code F1AP/f1ap_cu_rrc_message_transfer.c:L114–L128 checks DU UE ID matches stored mapping"]
    },
    {
      "name": "GTP-U/F1-U activation gated by valid DL UP TNL info",
      "result": "PASS",
      "evidence": ["Code LAYER2/nr_pdcp/cucp_cuup_handler.c:L340–L345 updates F1-U tunnel on explicit params", "No direct GTP start in DU UE Context handlers"]
    }
  ],
  "preliminary_verdict": "ambiguous",
  "rationale": "Specs require F1 Setup before UE-associated procedures and consistent IDs/Transaction ID. Code shows proper RRC transfer and ID checks and that GTP-U updates occur via bearer modification, not directly from DU context handlers. However, we did not find an explicit DU guard ensuring 'F1Ready' before handling UE CONTEXT MODIFICATION REQUEST, nor Transaction ID validation in that path. Without traces, it's unclear if a crafted sequence can bypass setup to trigger UP handling."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "F1AP §8.4.1.2 Initial UL RRC Message Transfer — Successful operation",
      "F1AP §9.3.1.23 Transaction ID",
      "F1AP §9.3.1.4 gNB-CU UE F1AP ID",
      "F1AP §9.3.1.5 gNB-DU UE F1AP ID"
    ],
    "code_citations": [
      "F1AP/f1ap_du_interface_management.c:L142–L160 (DU_send_F1_SETUP_REQUEST)",
      "F1AP/f1ap_du_rrc_message_transfer.c:L74–L100 (DU_send_INITIAL_UL_RRC_MESSAGE_TRANSFER)",
      "F1AP/f1ap_du_ue_context_management.c:L125–L137 (DU_handle_UE_CONTEXT_MODIFICATION_REQUEST)",
      "F1AP/f1ap_cu_rrc_message_transfer.c:L114–L128 (CU checks DU UE ID mapping)",
      "LAYER2/nr_pdcp/cucp_cuup_handler.c:L340–L345 (GtpuUpdateTunnelOutgoingAddressAndTeid)"
    ],
    "expected_vs_observed": "Expected: DU completes F1 Setup, initiates UE-associated logical F1 with Initial UL RRC, then processes UE Context Setup/Modification using consistent Transaction ID and bound CU/DU UE F1AP IDs before applying any UP TNL changes. Observed: report claims attacker sequence (forged Setup Success, mutated DU/Transaction IDs, early Initial UL RRC) leads DU to bypass setup and start GTP-U. Code suggests UP activation happens via bearer modification and explicit DL UP TNL parameters, but we did not find an explicit DU guard for 'F1Ready' or Transaction ID checks in the modification request path."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "ambiguous",
    "rationale": "Specs require F1 Setup prior to UE-associated procedures and consistent ID/Transaction binding. Code shows correct RRC transfer handling and ID checks at CU; UP tunnel updates are applied via bearer modification handlers. We did not find explicit DU-side guards ensuring F1 Setup is complete before accepting UE CONTEXT MODIFICATION REQUEST nor explicit Transaction ID validation in DU handlers. Without runtime traces, we cannot confirm the reported bypass or its impact on UP activation.",
    "citations": {
      "spec": [
        "F1AP §8.4.1.2 Initial UL RRC Message Transfer — Successful operation",
        "F1AP §9.3.1.23 Transaction ID",
        "F1AP §9.3.1.4 gNB-CU UE F1AP ID",
        "F1AP §9.3.1.5 gNB-DU UE F1AP ID"
      ],
      "code": [
        "F1AP/f1ap_du_interface_management.c:L142–L160",
        "F1AP/f1ap_du_rrc_message_transfer.c:L74–L100",
        "F1AP/f1ap_du_ue_context_management.c:L125–L137",
        "F1AP/f1ap_cu_rrc_message_transfer.c:L114–L128",
        "LAYER2/nr_pdcp/cucp_cuup_handler.c:L340–L345"
      ]
    },
    "suggested_repro": [
      "Establish SCTP; do NOT complete F1 Setup.",
      "Send UE CONTEXT SETUP REQUEST to CU; ensure rejection.",
      "Send forged UE CONTEXT SETUP RESPONSE/SUCCESS toward CU/DU as applicable.",
      "Send INITIAL UL RRC MESSAGE TRANSFER with mutated DU ID and Transaction ID.",
      "Attempt UE CONTEXT MODIFICATION REQUEST and observe if DU applies UP TNL or starts GTP-U."
    ],
    "risks": [
      "Potential UP activation without proper control-plane handshake",
      "UE context desynchronization at CU/DU",
      "Resource leakage by premature tunnel creation"
    ],
    "next_steps": [
      "Add explicit DU-side guard: reject UE-associated messages unless F1 Setup completed",
      "Validate Transaction ID continuity in DU handlers for context procedures",
      "Log and verify CU/DU UE F1AP ID binding on all context procedures",
      "Create unit/integration test to replay the crafted sequence and assert non-activation of UP"
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.